In [ ]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 45 kB/s 
     |████████████████████████████████| 199 kB 51.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=91e3bbe83210181f55369390f1c25053c46cd206240b650f92fb1e9add4eda64
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession 
spark = SparkSession.builder.appName('abc').getOrCreate()

In [ ]:
from pyspark.ml.classification import LogisticRegression

In [ ]:
input_data=spark.read.csv('/content/drive/MyDrive/ISEAR_pre_processes.csv',header=True,inferSchema=True)
 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
input_data.printSchema()

root
 |-- Emotion: string (nullable = true)
 |-- Clean_Text: string (nullable = true)



In [ ]:
input_data.count()

7447

In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

In [ ]:
tokenizer = Tokenizer(inputCol="Clean_Text", outputCol="words")
wordsData = tokenizer.transform(input_data)
wordsData.show(truncate=False)

+-------+----------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------+
|Emotion|Clean_Text                                                                                    |words                                                                                                        |
+-------+----------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------+
|joy    |days feel close partner friends feel peace experience close contact people regard greatly     |[days, feel, close, partner, friends, feel, peace, experience, close, contact, people, regard, greatly]      |
|fear   |time imagine love contact illness death                                                       |[time, imagine, love, contact, illne

In [ ]:
# hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=50000)
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures")
featurizedData = hashingTF.transform(wordsData)

idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)

rescaledData.show(truncate=False)

+-------+----------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Emotion|Clean_Text                                                                                    |words                                                                                                        |rawFeatures                                             

In [ ]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol="Emotion", outputCol="categoryIndex") 
indexed = indexer.fit(input_data).transform(rescaledData) 
indexed.show()

+-------+--------------------+--------------------+--------------------+--------------------+-------------+
|Emotion|          Clean_Text|               words|         rawFeatures|            features|categoryIndex|
+-------+--------------------+--------------------+--------------------+--------------------+-------------+
|    joy|days feel close p...|[days, feel, clos...|(262144,[24698,43...|(262144,[24698,43...|          0.0|
|   fear|time imagine love...|[time, imagine, l...|(262144,[22373,12...|(262144,[22373,12...|          2.0|
|  anger|obviously unjustl...|[obviously, unjus...|(262144,[17619,40...|(262144,[17619,40...|          1.0|
|sadness|think short time ...|[think, short, ti...|(262144,[14636,33...|(262144,[14636,33...|          5.0|
|disgust|gathering found i...|[gathering, found...|(262144,[8003,289...|(262144,[8003,289...|          3.0|
|  shame|realized directin...|[realized, direct...|(262144,[6017,331...|(262144,[6017,331...|          4.0|
|  guilt|feel guilty reali..

In [ ]:
from pyspark.sql.functions import col
df = indexed.select(col("features"),col("categoryIndex"))
df.show()

+--------------------+-------------+
|            features|categoryIndex|
+--------------------+-------------+
|(262144,[24698,43...|          0.0|
|(262144,[22373,12...|          2.0|
|(262144,[17619,40...|          1.0|
|(262144,[14636,33...|          5.0|
|(262144,[8003,289...|          3.0|
|(262144,[6017,331...|          4.0|
|(262144,[4254,153...|          6.0|
|(262144,[38004,66...|          0.0|
|(262144,[24980,10...|          2.0|
|(262144,[29744,71...|          1.0|
|(262144,[2437,343...|          5.0|
|(262144,[87405,16...|          3.0|
|(262144,[15391,12...|          4.0|
|(262144,[72176,10...|          6.0|
|(262144,[112473,1...|          0.0|
|(262144,[32983,68...|          2.0|
|(262144,[15623,11...|          1.0|
|(262144,[90370,10...|          5.0|
|(262144,[48448,66...|          3.0|
|(262144,[1696,675...|          4.0|
+--------------------+-------------+
only showing top 20 rows



In [ ]:
train,test=df.randomSplit([0.8, 0.2])
train.show()

+--------------------+-------------+
|            features|categoryIndex|
+--------------------+-------------+
|(262144,[86,32550...|          3.0|
|(262144,[134,6189...|          3.0|
|(262144,[135,6213...|          0.0|
|(262144,[150,9420...|          4.0|
|(262144,[150,1240...|          1.0|
|(262144,[150,1968...|          1.0|
|(262144,[150,2057...|          4.0|
|(262144,[150,3095...|          4.0|
|(262144,[150,6441...|          3.0|
|(262144,[161,1482...|          1.0|
|(262144,[285,6189...|          3.0|
|(262144,[329,1578...|          3.0|
|(262144,[329,3775...|          1.0|
|(262144,[329,3928...|          1.0|
|(262144,[329,6261...|          0.0|
|(262144,[329,6453...|          4.0|
|(262144,[329,6498...|          1.0|
|(262144,[329,8195...|          2.0|
|(262144,[329,1599...|          2.0|
|(262144,[329,1662...|          3.0|
+--------------------+-------------+
only showing top 20 rows



In [ ]:
model=LogisticRegression(labelCol='categoryIndex')   

In [ ]:
model=model.fit(train) 

In [ ]:
summary=model.summary

In [ ]:
summary.predictions.describe().show()

+-------+------------------+-----------------+
|summary|     categoryIndex|       prediction|
+-------+------------------+-----------------+
|  count|              5921|             5921|
|   mean|2.9888532342509713|2.989697686201655|
| stddev| 2.000813361361693|2.000184601229678|
|    min|               0.0|              0.0|
|    max|               6.0|              6.0|
+-------+------------------+-----------------+



In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator 

In [ ]:
prediction=model.evaluate(test)

In [ ]:
prediction.predictions.show()

+--------------------+-------------+--------------------+--------------------+----------+
|            features|categoryIndex|       rawPrediction|         probability|prediction|
+--------------------+-------------+--------------------+--------------------+----------+
|(262144,[119,3506...|          2.0|[222.821352215428...|[3.61037477674856...|       2.0|
|(262144,[134,2906...|          3.0|[228.367187955144...|[9.26257292673363...|       1.0|
|(262144,[134,4267...|          3.0|[-296.97107619456...|[0.0,1.0899415956...|       6.0|
|(262144,[134,8402...|          3.0|[113.224845179144...|[1.85301342015778...|       1.0|
|(262144,[135,6240...|          2.0|[327.427567922713...|[1.0,3.2415213023...|       0.0|
|(262144,[329,1243...|          6.0|[-136.67606232840...|[6.62843006432149...|       1.0|
|(262144,[329,1789...|          6.0|[-603.78642027951...|[0.0,0.0,0.0,0.0,...|       6.0|
|(262144,[329,3095...|          6.0|[647.291948586586...|[1.0,1.9025523632...|       0.0|
|(262144,[

In [ ]:
evaluator=MulticlassClassificationEvaluator(predictionCol='prediction',labelCol='categoryIndex')

In [ ]:
evaluator.evaluate(prediction.predictions)

0.4057984619687023

In [ ]:
from pyspark.ml.classification import NaiveBayes 


In [ ]:
rf = NaiveBayes(modelType='multinomial',featuresCol = 'features', labelCol = 'categoryIndex')


In [ ]:
rfModel = rf.fit(train)


In [ ]:
prediction = rfModel.transform(test)


In [ ]:
evaluator1=MulticlassClassificationEvaluator(predictionCol='prediction',labelCol='categoryIndex')

In [ ]:
nbaccuracy = evaluator.evaluate(prediction) 
print("Test accuracy = " + str(nbaccuracy))

Test accuracy = 0.48714168292389465
